# Key Considerations for Your Dataproc Cluster

## 1. Cluster Resources:

-   **Master**: `n2-standard-4` (4 vCPUs, 16 GB RAM, 32GB disk)
-   **Workers (2x)**: `n2-standard-4` (4 vCPUs, 16 GB RAM, 64GB disk
    each)
-   **Total**: 8 worker vCPUs, \~32 GB RAM (excluding master node)

## 2. Dataproc Features Disabled:

-   No **autoscaling**, **Metastore**, **advanced execution layer**,
    **advanced optimizations**
-   **Storage**: `pd-balanced` (no SSDs, so I/O optimization is crucial)
-   **Networking**: Internal IP **enabled**

## 3. Optimization Strategy:

-   Tune **shuffle partitions**, **broadcast join threshold**, and
    **storage persistence**
-   Adjust **parallelism** based on 2 workers x 4 cores
-   Avoid **excessive caching** due to disk-based storage


https://spark.apache.org/docs/latest/configuration.html


In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Imputer
from pyspark.sql.window import *

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Module 4-Spark_configuration_optimization') \
    .config('spark.executor.memory', '6g') \
    .config('spark.executor.cores', '4') \
    .config('spark.executor.instances', '2') \
    .config('spark.driver.memory', '4g') \
    .config('spark.driver.maxResultSize', '2g') \
    .config('spark.sql.shuffle.partitions', '64') \
    .config('spark.default.parallelism', '64') \
    .config('spark.sql.adaptive.enabled', 'true') \
    .config('spark.sql.adaptive.coalescePartitions.enabled', 'true') \
    .config('spark.sql.autoBroadcastJoinThreshold', 50*1024*1024) \
    .config('spark.sql.files.maxPartitionBytes', '64MB') \
    .config('spark.sql.files.openCostInBytes', '2MB') \
    .config('spark.memory.fraction', 0.8) \
    .config('spark.memory.storageFraction', 0.2) \
    .getOrCreate()


25/08/27 21:43:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark

In [4]:
hdfs_path= '/tmp/Brazilian'

In [5]:
customers_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_customers_dataset.csv')

geolocation_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_geolocation_dataset.csv')

order_items_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_items_dataset.csv')

order_payments_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_payments_dataset.csv')

order_reviews_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_order_reviews_dataset.csv')

orders_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_orders_dataset.csv')

products_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_products_dataset.csv')

sellers_df  = spark.read\
.format('csv')\
.option('header',True)\
.option('inferSchema',True)\
.load(hdfs_path+'/olist_sellers_dataset.csv')


25/08/27 21:43:55 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/27 21:44:10 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/27 21:44:25 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/27 21:44:40 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/08/27 21:44:55 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [6]:
!hadoop fs -ls -h /tmp/olist_processed


Found 12 items
-rw-r--r--   2 root hadoop          0 2025-08-27 20:28 /tmp/olist_processed/_SUCCESS
-rw-r--r--   2 root hadoop      9.5 M 2025-08-27 20:27 /tmp/olist_processed/part-00000-59d041a6-ebd1-411c-bc21-b203436d348f-c000.snappy.parquet
-rw-r--r--   2 root hadoop      9.1 M 2025-08-27 20:27 /tmp/olist_processed/part-00001-59d041a6-ebd1-411c-bc21-b203436d348f-c000.snappy.parquet
-rw-r--r--   2 root hadoop      9.6 M 2025-08-27 20:27 /tmp/olist_processed/part-00002-59d041a6-ebd1-411c-bc21-b203436d348f-c000.snappy.parquet
-rw-r--r--   2 root hadoop      9.4 M 2025-08-27 20:27 /tmp/olist_processed/part-00003-59d041a6-ebd1-411c-bc21-b203436d348f-c000.snappy.parquet
-rw-r--r--   2 root hadoop     11.4 M 2025-08-27 20:27 /tmp/olist_processed/part-00004-59d041a6-ebd1-411c-bc21-b203436d348f-c000.snappy.parquet
-rw-r--r--   2 root hadoop     11.1 M 2025-08-27 20:27 /tmp/olist_processed/part-00005-59d041a6-ebd1-411c-bc21-b203436d348f-c000.snappy.parquet
-rw-r--r--   2 root hadoop     10.5 

In [7]:
#reading the above prequest file and parquet only cause it is optimized with spark
full_orders_df = spark.read.parquet('/tmp/olist_processed')
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

# Optimized Join Quires And Stratigies

# Broadcast

In [8]:
customer_broadcast_df = broadcast(customers_df)
optimized_broadcast_join = full_orders_df.join(customer_broadcast_df,"customer_id")

# Sort and merg join

In [9]:
sorted_customers_df = customers_df.sortWithinPartitions('customer_id')
sorted_orders_df= full_orders_df.sortWithinPartitions('customer_id') 

optimized_merged_full_orders_df = sorted_orders_df.join(sorted_customers_df,'customer_id')

# Bucket join 

In [10]:
bucketed_customers_df = customers_df.repartition(10,'customer_id')
bucketed_orders_df= full_orders_df.repartition(10,'customer_id') 

bucketed_join_df  = bucketed_orders_df.join(bucketed_customers_df,'customer_id') 

# Skew join handling 

In [11]:
skew_handeled_join = full_orders_df.join(customers_df.hint("SKEW"),'customer_id')
#### this is not supported on the older veresions of Spark so make sure that ur version supporrts skew joins 

25/08/27 21:45:33 WARN HintErrorLogger: Unrecognized hint: SKEW()


# Serving Data

## Objective:

- Save and retrieve processed data efficiently inside Dataproc.
- Serve data in a structured way for analysis.
- Use Parquet, Hive, and CSV



In [12]:
!hadoop fs -ls /tmp/

Found 20 items
drwxr-xr-x   - harshvardhansahu06733 hadoop          0 2025-08-23 16:56 /tmp/Brazilian
drwxr-xr-x   - root                  hadoop          0 2025-08-07 21:19 /tmp/External_table_data
drwxr-xr-x   - root                  hadoop          0 2025-06-29 21:17 /tmp/aciteve_city_300.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop    1060750 2025-06-24 12:30 /tmp/customers_1.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop   10528211 2025-06-24 12:30 /tmp/customers_10.csv
-rw-r--r--   2 root                  hadoop 1259828690 2025-07-06 20:02 /tmp/customers_1100.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop  168541068 2025-06-24 12:30 /tmp/customers_150.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop  343317147 2025-06-24 12:31 /tmp/customers_300.csv
-rw-r--r--   2 harshvardhansahu06733 hadoop        351 2025-07-07 19:07 /tmp/customers_data_with_errors.csv
-rw-r--r--   2 root                  hadoop        209 2025-07-28 20:22 /tmp/dates_data.csv
-rw-r--r--   2 root           

In [13]:
#save as parquet in hdfs
full_orders_df.write.mode('overwrite').parquet('/tmp/olist_processed_module4')

25/08/27 21:45:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [14]:
# save it as a parquet in a google cloud storage 

In [15]:
full_orders_df.write.mode('overwrite').parquet('gs://dataproc-staging-asia-south1-405952174203-miyokyzu/temp_data') 

In [16]:
# to save as table we have got save as table function 

In [17]:
full_orders_df.write.mode('overwrite').saveAsTable('full_order_detail')

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
25/08/27 21:51:12 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [19]:
spark.sql('show tables').show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default| cleaned_products|      false|
|  default|   customers_1100|      false|
|  default| external_table_2|      false|
|  default|full_order_detail|      false|
+---------+-----------------+-----------+



In [20]:
# for saving it as csv file 

In [24]:
!hadoop fs -mkdir /tmp/olist_csv_processed/

mkdir: `/tmp/olist_csv_processed': File exists


In [25]:
full_orders_df.write.mode('overwrite').option('header','true').csv('/tmp/olist_csv_processed/')

25/08/27 21:57:09 WARN TaskSetManager: Lost task 0.0 in stage 23.0 (TID 57) (testcluster1-w-0.asia-south1-b.c.midyear-guild-461710-g9.internal executor 2): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://testcluster1-m/tmp/olist_csv_processed.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:775)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:493)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.sp

Py4JJavaError: An error occurred while calling o200.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 4 times, most recent failure: Lost task 0.3 in stage 23.0 (TID 63) (testcluster1-w-0.asia-south1-b.c.midyear-guild-461710-g9.internal executor 2): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://testcluster1-m/tmp/olist_csv_processed.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:775)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:493)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.sca